<a href="https://colab.research.google.com/github/RomanGustavo/Mestrado----Projeto-Carro/blob/main/TWICE_Colab_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TWICE Dataset — Colab Starter
Este notebook prepara o ambiente, baixa o **subset** do TWICE, lê `.osi`, projeta cuboids (via utilitários do repo) e mostra como visualizar radar e rodar um detector (YOLOv7) para DEMO.


CEL 1 — instalar pacotes

o que faz: pip install de dependências usadas ao longo do notebook:

opencv-python (ler/decodificar imagens JPEG dentro dos .osi, fazer vídeo),

tqdm (barrinhas de progresso),

open3d (visualizar/grav ar nuvem de pontos .ply — útil pro LiDAR),

protobuf==4.25.3 (compatível com as mensagens OSI),

matplotlib (gráficos simples).

relação com o TWICE: os arquivos do TWICE usam OSI via protobuf; sem o protobuf correto e o OpenCV, você não abre os frames da câmera nem gera os vídeos.

In [6]:
# CEL 1 — pacotes base
!pip -q install --upgrade opencv-python tqdm open3d protobuf==4.25.3 matplotlib

# Forçar protobuf 4 só para os scripts OSI/TWICE
!pip install "protobuf==4.25.3" --force-reinstall

# Depois reinstalar o grpcio-status na versão compatível com protobuf 4
!pip install "grpcio-status==1.60.0"

  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.3 which is incompatible.


CEL 2 — instalar/ativar o OSI (Open Simulation Interface)

o que faz: tenta import osi3. se não existir, clona o repositório oficial do OSI, gera os módulos python a partir dos .proto (com setup.py build_py) e instala com pip.

relação com o TWICE: os .osi do TWICE são mensagens protobuf do OSI (classes como SensorData e SensorDataSeries). sem o pacote osi3, você não consegue decodificar o conteúdo dos arquivos de câmera/radar/lidar/IMU.

In [7]:
# CEL 2 — Open Simulation Interface (OSI): tenta importar; se não existir, compila a partir do repo oficial
try:
    import osi3
    print("✅ osi3 já disponível")
except Exception as e:
    print("⚠️ osi3 não encontrado. Clonando OSI e instalando a partir do código-fonte...")
    !git clone -q https://github.com/OpenSimulationInterface/open-simulation-interface.git
    %cd /content/open-simulation-interface
    !python3 setup.py -q build_py
    !pip -q install .
    %cd /content
    import osi3
    print("✅ osi3 instalado")

✅ osi3 já disponível


CEL 3 — clonar os scripts do repositório TWICE

o que faz: git clone de TWICEdataset/scripts e entra na pasta.

relação com o TWICE: esse repo traz utilitários prontos que já entendem a estrutura OSI/OpenLABEL do TWICE:

cuboid_project.py: projeção de cuboids 3D → imagem (usa intrínsecas/extrínsecas).

radar_osi_plot.py: leitura/plot do radar (modos cluster e object list) + trajetória.

lidar_osi2PLY.py: converter LiDAR .osi → .ply.

tutorial.ipynb: exemplos prontos.

In [8]:
# CEL 3 — clonar scripts
!git clone -q https://github.com/TWICEdataset/scripts.git
%cd /content/scripts
!ls -la

/content/scripts
total 136
drwxr-xr-x 6 root root  4096 Aug 24 14:56 .
drwxr-xr-x 1 root root  4096 Aug 24 14:15 ..
-rw-r--r-- 1 root root 16496 Aug 24 14:15 cuboid_project.py
-rw-r--r-- 1 root root  4891 Aug 24 14:15 file_selector.py
drwxr-xr-x 8 root root  4096 Aug 24 14:15 .git
-rw-r--r-- 1 root root  3317 Aug 24 14:15 lidar_osi2PLY.py
drwxr-xr-x 2 root root  4096 Aug 24 14:15 __pycache__
-rw-r--r-- 1 root root 27662 Aug 24 14:15 radar_osi_plot.py
-rw-r--r-- 1 root root  1545 Aug 24 14:15 README.md
-rw-r--r-- 1 root root   117 Aug 24 14:15 requirements.txt
drwxr-xr-x 2 root root  4096 Aug 24 14:15 Results
drwxr-xr-x 5 root root  4096 Aug 24 14:56 scripts
-rw-r--r-- 1 root root 18228 Aug 24 14:15 tutorial.ipynb
-rw-r--r-- 1 root root 15554 Aug 24 14:15 TWICE_df.csv
-rw-r--r-- 1 root root    20 Aug 24 14:15 TWICE_path.txt
-rw-r--r-- 1 root root  2350 Aug 24 14:15 vehicle_to_image.py


CEL 4 — baixar e extrair o subset do dataset

o que faz: baixa TWICEsubset.zip (~439 MB) do servidor e extrai em /content/TWICE.

CEL 4 “robutizada”

urllib + certifi (mantém validação TLS)

wget com --no-check-certificate (ignora verificação se 1 falhar)

curl -k (fallback final, também ignora verificação)

Depois ela verifica o arquivo, mostra tamanho e extrai

relação com o TWICE: você passa a ter localmente a árvore de cenários do dataset (ex.: Scenarios/dynamic ego/.../real/daytime/.../{Camera,Radar,LiDAR,IMU ...}), com arquivos .osi por sensor e arquivos OpenLABEL .json (anotações e calibrações).

In [11]:
# CEL 4 (robusta) — baixar o subset e extrair, contornando SSL do servidor
import os, zipfile, pathlib, hashlib, sys

subset_url = "https://twice.eletrica.ufpr.br/TWICEsubset.zip"
zip_path = "/content/TWICEsubset.zip"
out_dir = pathlib.Path("/content/TWICE")
out_dir.mkdir(parents=True, exist_ok=True)

def sha256sum(path):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1<<20), b""):
            h.update(chunk)
    return h.hexdigest()

ok = False
if not os.path.exists(zip_path):
    print("⬇️ Tentando baixar com urllib + certifi (TLS verificado)...")
    try:
        import ssl, certifi, urllib.request
        ctx = ssl.create_default_context(cafile=certifi.where())
        opener = urllib.request.build_opener(urllib.request.HTTPSHandler(context=ctx))
        urllib.request.install_opener(opener)
        urllib.request.urlretrieve(subset_url, zip_path)
        ok = True
        print("✅ Baixado com validação TLS (urllib+certifi).")
    except Exception as e:
        print("❌ Falhou urllib+certifi:", e)

    if not ok:
        print("⬇️ Tentando com wget (sem checar certificado)...")
        rc = os.system(f'wget -O "{zip_path}" --no-check-certificate "{subset_url}"')
        ok = (rc == 0)
        print("✅ Baixado com wget." if ok else "❌ wget falhou.")

    if not ok:
        print("⬇️ Tentando com curl (sem checar certificado)...")
        rc = os.system(f'curl -L -k "{subset_url}" -o "{zip_path}"')
        ok = (rc == 0)
        print("✅ Baixado com curl." if ok else "❌ curl falhou.")

else:
    ok = True
    print("📦 ZIP já existe. Pulando download.")

if not ok:
    print("\n⚠️ Não foi possível baixar automaticamente.")
    print("Opções alternativas:")
    print("  (a) Faça upload manual de TWICEsubset.zip na raiz (/content) e rode esta célula de novo; ou")
    print("  (b) Monte o Google Drive e copie o arquivo para /content:")
    print('      from google.colab import drive; drive.mount("/content/drive")')
    print('      !cp "/content/drive/MyDrive/TWICEsubset.zip" "/content/TWICEsubset.zip"')
    raise SystemExit(1)

# Verificação rápida e extração
size_mb = os.path.getsize(zip_path) / (1024*1024)
print(f"📏 Tamanho do ZIP: {size_mb:.1f} MB")
print("🔐 SHA256:", sha256sum(zip_path)[:16], "…")  # fingerprint curto

print("📦 Extraindo…")
with zipfile.ZipFile(zip_path, "r") as zf:
    bad = zf.testzip()
    if bad:
        raise RuntimeError(f"Arquivo corrompido (primeiro erro em: {bad})")
    zf.extractall(out_dir)

print("✅ Pronto. Dados em:", out_dir)


⬇️ Tentando baixar com urllib + certifi (TLS verificado)...
❌ Falhou urllib+certifi: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1010)>
⬇️ Tentando com wget (sem checar certificado)...
✅ Baixado com wget.
📏 Tamanho do ZIP: 438.5 MB
🔐 SHA256: b3469614e3e95491 …
📦 Extraindo…
✅ Pronto. Dados em: /content/TWICE


CEL 5 — configurar caminhos e importar utilitários

o que faz: define DATA_ROOT=/content/TWICE e adiciona /content/scripts no sys.path. depois importa:

rotation_matrix e cuboid_corners (para compor as arestas do cubo 3D),

Camera (modelo de câmera usado na projeção).

relação com o TWICE: prepara o ambiente para projetar as anotações 3D (OpenLABEL) sobre os frames de câmera do TWICE e para acessar outros utilitários do repo.

In [12]:
# CEL 5 — paths e imports
import sys, pathlib
DATA_ROOT = pathlib.Path("/content/TWICE")
SCRIPTS_ROOT = pathlib.Path("/content/scripts")
sys.path.insert(0, str(SCRIPTS_ROOT))

print("DATA_ROOT =", DATA_ROOT)
print("SCRIPTS_ROOT =", SCRIPTS_ROOT)

from cuboid_project import rotation_matrix, cuboid_corners
from vehicle_to_image import Camera
print("✅ utilitários importados")

DATA_ROOT = /content/TWICE
SCRIPTS_ROOT = /content/scripts
✅ utilitários importados


CEL 6 — ler frames de câmera de um arquivo .osi e gerar um vídeo preview

o que faz (técnico):

encontra uma pasta .../Camera/ dentro do subset.

abre o .osi da câmera em modo binário e faz o loop:

lê 4 bytes com struct.unpack("I", ...) → isso é o tamanho da próxima mensagem protobuf,

lê o próximo blob de bytes (msg_len),

instancia SensorData() (classe OSI) e chama ParseFromString(msg),

obtém sd.image_data[0].image.raw_data (bytes JPEG do frame),

decodifica para numpy com cv2.imdecode(...).

guarda ~150 frames e salva um vídeo .mp4 (15 fps) com cv2.VideoWriter.

-- + robusto:

Lê cada mensagem protobuf do .osi (câmera),

Percorre recursivamente os campos e submensagens,

Detecta bytes com “assinatura” de JPEG (FF D8 … FF D9) ou PNG (89 50 4E 47),

Decodifica com OpenCV e gera o preview MP4.

-- Solução do erro:
Esse arquivo é camera_sv_… (o “sv” costuma indicar SensorView, não SensorData).

Em TWICE, câmera/LiDAR foram “empacotados” como mensagens individuais dentro do .osi (loop [len][protobuf]), e algumas builds do OSI mudam os nomes/caminhos dos campos.

Para resolver:

Tentar SensorData, depois SensorView (duas classes diferentes do OSI).

Se ainda não achar, varrer o blob inteiro procurando JPEG/PNG diretamente (SOI/EOI), e decodificar.

relação com o TWICE: você reconstrói as imagens da câmera a partir do arquivo OSI do TWICE (cada mensagem é um frame), e cria um preview para inspecionar rapidamente se o caso/condição climática está ok.

In [15]:
# CEL 6 — iterar frames do .osi (SensorData → SensorView → varredura do blob) e salvar um MP4
import cv2, struct
import numpy as np
from pathlib import Path

# 1) Tentar importar classes OSI
SensorData = None
SensorView = None
try:
    from cuboid_project import SensorData as TW_SensorData
    SensorData = TW_SensorData
except:
    pass

try:
    # alguns pacotes expõem diretamente
    import osi3
    if SensorData is None and hasattr(osi3, "SensorData"):
        SensorData = osi3.SensorData
    # SensorView pode vir como atributo ou via módulo pb2
    if hasattr(osi3, "SensorView"):
        SensorView = osi3.SensorView
    else:
        try:
            from osi3 import sensorview_pb2
            SensorView = sensorview_pb2.SensorView
        except:
            pass
except:
    pass

JPEG_SOI = b"\xFF\xD8"
JPEG_EOI = b"\xFF\xD9"
PNG_SIG  = b"\x89PNG\r\n\x1a\n"

def looks_like_img(buf: bytes) -> bool:
    return (
        (len(buf) > 8 and buf.startswith(JPEG_SOI) and buf.endswith(JPEG_EOI)) or
        (len(buf) > 8 and buf.startswith(PNG_SIG)) or
        (len(buf) > 8 and buf.startswith(JPEG_SOI) and b"\xFF\xD9" in buf[-4:])
    )

def scan_blob_for_image(blob: bytes):
    # Procura um JPEG (SOI...EOI)
    start = blob.find(JPEG_SOI)
    if start != -1:
        end = blob.find(JPEG_EOI, start+2)
        if end != -1:
            return blob[start:end+2]
    # Procura um PNG
    start = blob.find(PNG_SIG)
    if start != -1:
        # PNG é chunked; tentamos até o fim do blob
        return blob[start:]
    return None

def extract_img_from_any_message(msg):
    # Percorre recursivamente a mensagem protobuf atrás de campos bytes que pareçam imagem
    try:
        fields = msg.ListFields()
    except Exception:
        return None
    from google.protobuf.descriptor import FieldDescriptor
    for desc, val in fields:
        try:
            if desc.type == FieldDescriptor.TYPE_BYTES and looks_like_img(val):
                return val
        except Exception:
            pass
        # Submensagem
        if hasattr(val, "ListFields"):
            out = extract_img_from_any_message(val)
            if out is not None:
                return out
        # Lista de submensagens
        if isinstance(val, (list, tuple)):
            for elem in val:
                if hasattr(elem, "ListFields"):
                    out = extract_img_from_any_message(elem)
                    if out is not None:
                        return out
    return None

def iter_camera_frames(osi_path: Path):
    with open(osi_path, "rb") as f:
        while True:
            len_bytes = f.read(4)
            if not len_bytes:
                break
            (msg_len,) = struct.unpack("I", len_bytes)
            blob = f.read(msg_len)

            # a) tentar SensorData
            if SensorData is not None:
                try:
                    sd = SensorData()
                    sd.ParseFromString(blob)
                    img_bytes = extract_img_from_any_message(sd)
                    if img_bytes:
                        img_np = np.frombuffer(img_bytes, dtype=np.uint8)
                        frame = cv2.imdecode(img_np, cv2.IMREAD_COLOR)
                        if frame is not None:
                            yield frame
                            continue
                except Exception:
                    pass

            # b) tentar SensorView
            if SensorView is not None:
                try:
                    sv = SensorView()
                    sv.ParseFromString(blob)
                    img_bytes = extract_img_from_any_message(sv)
                    if img_bytes:
                        img_np = np.frombuffer(img_bytes, dtype=np.uint8)
                        frame = cv2.imdecode(img_np, cv2.IMREAD_COLOR)
                        if frame is not None:
                            yield frame
                            continue
                except Exception:
                    pass

            # c) fallback bruto: vasculhar o blob por JPEG/PNG
            img_bytes = scan_blob_for_image(blob)
            if img_bytes:
                img_np = np.frombuffer(img_bytes, dtype=np.uint8)
                frame = cv2.imdecode(img_np, cv2.IMREAD_COLOR)
                if frame is not None:
                    yield frame
                    continue
            # Se chegou aqui, essa mensagem não tinha imagem (ou formato não suportado). Continua.

# Localizar o arquivo de Câmera (você já tinha cam_file)
scenario_dir = next(DATA_ROOT.rglob("Camera"), None)
assert scenario_dir is not None, "Não encontrei /Camera no subset. Verifique a extração do ZIP."
cam_file = next(Path(scenario_dir).glob("*.osi"))
print("🎥 Câmera .osi:", cam_file)

frames = []
for i, fr in enumerate(iter_camera_frames(cam_file)):
    frames.append(fr)
    if i >= 150:
        break

if not frames:
    raise RuntimeError("Ainda não consegui extrair frames. Rode a célula de DIAGNÓSTICO para inspecionar os campos.")

h, w = frames[0].shape[:2]
out_path = "/content/camera_preview.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
vw = cv2.VideoWriter(out_path, fourcc, 15, (w, h))
for fr in frames:
    vw.write(fr)
vw.release()
print(f"✅ Preview salvo em: {out_path}  ({len(frames)} frames)")


🎥 Câmera .osi: /content/TWICE/TWICEsubset/Scenarios/dynamic_ego/CCRb/real/daytime/cluster/test_run_1/Camera/camera_sv_350_300.osi
✅ Preview salvo em: /content/camera_preview.mp4  (151 frames)


CEL 7 — carregar OpenLABEL e preparar a projeção 3D→2D

o que faz:

abre o *open*label*camera*.json do cenário,

extrai a matriz intrínseca K a partir de camera_matrix_3x4 → remodela para (3×4) e usa as 3×3 primeiras colunas,

(na célula de exemplo) só escreve um texto no frame; para desenhar as arestas do cubo você usa:

os “cuboids” (posição, dimensões, orientação) em openlabel["objects"][...]["object_data"]["cuboid"]["val"],

funções do cuboid_project.py para obter os 8 cantos 3D (cuboid_corners), aplicar rotação/translação (extrínsecas) e projetar com K (intrínsecas),

desenhar arestas com cv2.line.

-- Solução: isso acontece porque alguns arquivos OpenLABEL do TWICE não usam a chave intrinsics_pinhole (ela aparece no exemplo do artigo, mas no subset real pode estar com outro nome/caminho).

CEL 7 substituta, mais robusta, que:

carrega o JSON do OpenLABEL,

nesse JSON do OpenLABEL as intrínsecas não estão gravadas (só há timestamp e Streams). Sem dramas: dá pra seguir de duas formas:

Procurar mais fundo dentro de Streams.Camera1 (às vezes a matriz aparece com outro nome/caminho).

Fallback físico: estimar K a partir do FOV + tamanho da imagem. O artigo informa ~60° (horizontal) e ~36.6° (vertical) para a Sekonix 1920×1232 — isso é suficiente para projetar cubos com boa aproximação.

procura recursivamente por matrizes intrínsecas em formatos comuns:

camera_matrix_3x4 (tira a parte 3×3),

K (3×3),

campos soltos fx, fy, cx, cy,

outras variações como camera_matrix, intrinsics, etc.

imprime qual caminho foi encontrado,

desenha um texto no primeiro frame (só para confirmar; depois a gente pluga no desenho 3D→2D).

relação com o TWICE: o TWICE rotula em OpenLABEL (padrão da ASAM): você aproveita essas anotações para desenhar ground-truth sobre os frames, checar calibração e gerar vídeos anotados.

In [19]:
# CEL 7 (robusta v2) — localizar K em Streams.Camera1 OU estimar por FOV (fallback)
import json, cv2, numpy as np, math
from pathlib import Path

openlabel_file = next(Path(scenario_dir).glob("*open*label*camera*.json"), None)
print("🧾 OpenLABEL:", openlabel_file)

with open(openlabel_file, "r") as f:
    ol = json.load(f)

# 1) pega o nó Camera1 (se existir)
cam1 = {}
try:
    cam1 = ol["openlabel"]["frames"][0]["frame_properties"]["Streams"]["Camera1"]
except Exception:
    cam1 = {}

def _flatten(d, prefix=""):
    if isinstance(d, dict):
        for k,v in d.items():
            p = f"{prefix}.{k}" if prefix else k
            yield from _flatten(v, p)
    elif isinstance(d, list):
        for i,v in enumerate(d):
            p = f"{prefix}[{i}]"
            yield from _flatten(v, p)
    else:
        yield prefix, d

def _find_K_in_dict(node):
    # procura 3x4 -> 3x3
    for path, val in _flatten(node):
        if isinstance(val, list) and len(val)==12 and all(isinstance(x,(int,float)) for x in val):
            if any(tok in path.lower() for tok in ["camera_matrix_3x4","matrix_3x4","3x4","pinhole"]):
                K = np.array(val, dtype=float).reshape(3,4)[:,:3]
                return K, f"{path} (3x4→K)"
    # procura 3x3 direta
    for path, val in _flatten(node):
        if isinstance(val, list) and len(val)==9 and all(isinstance(x,(int,float)) for x in val):
            if any(tok in path.lower() for tok in ["camera_matrix","intrinsics","k","matrix_3x3","3x3"]):
                K = np.array(val, dtype=float).reshape(3,3)
                return K, f"{path} (3x3)"
    # fx,fy,cx,cy no mesmo nível
    def _fxfy_from_dict(d):
        if not isinstance(d, dict): return None
        keys = {k.lower():k for k in d.keys()}
        need = ["fx","fy","cx","cy"]
        if all(n in keys for n in need):
            try:
                fx = float(d[keys["fx"]]); fy = float(d[keys["fy"]])
                cx = float(d[keys["cx"]]); cy = float(d[keys["cy"]])
                K  = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]],dtype=float)
                return K
            except: return None
        return None
    def _walk(d, path=""):
        if isinstance(d, dict):
            K = _fxfy_from_dict(d)
            if K is not None:
                return K, f"{path or 'fx/fy/cx/cy'}"
            for k,v in d.items():
                out = _walk(v, f"{path}.{k}" if path else k)
                if out: return out
        elif isinstance(d, list):
            for i,v in enumerate(d):
                out = _walk(v, f"{path}[{i}]")
                if out: return out
        return None
    out = _walk(node)
    if out: return out
    return None, None

K, where = _find_K_in_dict(cam1)

if K is None:
    # 2) Fallback: estimar K por FOV (do paper) + tamanho do frame
    # pegue dimensões do frame já decodificado na CEL 6
    H, W = frames[0].shape[:2]
    # Tenta ler FOV do JSON (se existir); caso contrário, usa nominal 60° × 36.6°
    hfov_deg = 60.0
    vfov_deg = 36.6
    # procure possíveis campos de FOV:
    for path,val in _flatten(cam1):
        if isinstance(val,(int,float)) and "fov" in path.lower():
            # heurística: se encontrar dois floats, assume primeiro horizontal e segundo vertical
            if "horizontal" in path.lower(): hfov_deg = float(val)
            if "vertical"   in path.lower(): vfov_deg = float(val)
    # converte para radianos
    hfov = math.radians(hfov_deg)
    vfov = math.radians(vfov_deg)
    # modelo pinhole: fx = (W/2)/tan(hfov/2); fy = (H/2)/tan(vfov/2)
    fx = (W/2.0) / math.tan(hfov/2.0)
    fy = (H/2.0) / math.tan(vfov/2.0)
    cx = W/2.0
    cy = H/2.0
    K = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]], dtype=float)
    where = f"fallback FOV ({hfov_deg:.1f}° × {vfov_deg:.1f}°) + frame {W}×{H}"

print("✅ K usada:", where)
print(K)

# Marcação simples no primeiro frame só para confirmar
frame0 = frames[0].copy()
cv2.putText(frame0, f"K: {where}", (30,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
cv2.imwrite("/content/cuboids_demo.jpg", frame0)
print("🖼️ Exemplo salvo em /content/cuboids_demo.jpg")


🧾 OpenLABEL: /content/TWICE/TWICEsubset/Scenarios/dynamic_ego/CCRb/real/daytime/cluster/test_run_1/Camera/open_label_camera.json
✅ K usada: fallback FOV (60.0° × 36.6°) + frame 1920×1208
[[1.66276878e+03 0.00000000e+00 9.60000000e+02]
 [0.00000000e+00 1.82632728e+03 6.04000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
🖼️ Exemplo salvo em /content/cuboids_demo.jpg


CEL 8 — visualizar radar (cluster / object list) e trajetórias

o que faz: importa radar_osi_plot.py, localiza um .osi de radar (se existir no subset daquele cenário) e orienta usar as funções do script para plotar:

cluster: nuvem de pontos (cada reflexão do radar),

object list: lista de objetos agregados pelo ECU (com velocidade/posição/RCS),

posição do ego e do alvo (IMU/GNSS) ao longo do tempo.

relação com o TWICE: permite ver como o radar “enxerga” o mesmo cenário da câmera (real e HIL), inclusive comparando modos de operação.

In [20]:
# CEL 8 — visualizar radar (cluster/object list) — precisa de arquivo em /Radar
from importlib import reload
import os
try:
    import radar_osi_plot as rplt
    reload(rplt)
    radar_dir = (Path(scenario_dir).parent / "Radar")
    if radar_dir.exists():
        radar_osi = next(radar_dir.glob("*.osi"))
        print("📡 Radar .osi:", radar_osi)
        # Abra o arquivo radar_osi_plot.py para ver como salvar vídeo (há exemplo lá).
    else:
        print("⚠️ Este caso do subset pode não ter Radar. Teste outro cenário ou use o dataset completo.")
except Exception as e:
    print("⚠️ Import do script de radar falhou:", e)

📡 Radar .osi: /content/TWICE/TWICEsubset/Scenarios/dynamic_ego/CCRb/real/daytime/cluster/test_run_1/Radar/radar_sd_350_300.osi


CEL 9 — (opcional) rodar YOLOv7 como demo

o que faz: exporta ~30 frames do preview para /content/frames/*.jpg, clona o repositório do YOLOv7 e roda detect.py com yolov7.pt. salva imagens e labels preditos em /content/y7out/det.

instala PyTorch já compilado (CUDA 12.1, CPU fallback), clona o YOLOv7 e roda o detect.py.

relação com o TWICE: demonstra um baseline de detecção sobre os frames do TWICE. a partir daí, você pode comparar com GT do OpenLABEL (convertendo cuboids 3D → caixas 2D) para calcular IoU/AP, como o artigo exemplifica.

In [22]:
# === YOLOv7 - instalação robusta e inferência nos frames ===
import os, cv2, pathlib, sys, subprocess, textwrap

# 0) Garantir que estamos na raiz do Colab
%cd /content

# 1) Clonar repositório yolov7 (se ainda não existe)
if not pathlib.Path("/content/yolov7").exists():
    !git clone -q https://github.com/WongKinYiu/yolov7.git
else:
    print("🔁 yolov7 já clonado.")

# 2) Instalar PyTorch pré-compilado (tenta CUDA 12.1; se falhar, cai para CPU)
def install_torch():
    try:
        print("⬇️ Instalando PyTorch + TorchVision (CUDA 12.1 wheels)...")
        !pip -q install --upgrade "torch>=2.2,<3" "torchvision>=0.17,<0.20" --index-url https://download.pytorch.org/whl/cu121
    except Exception as e:
        print("⚠️ Falhou CUDA 12.1, tentando CPU wheels…")
        !pip -q install --upgrade "torch>=2.2,<3" "torchvision>=0.17,<0.20" --index-url https://download.pytorch.org/whl/cpu
install_torch()

# 3) Instalar dependências do YOLOv7 (ignorando torch, que já instalamos acima)
%cd /content/yolov7
# remove linhas de torch do requirements (se houver) para evitar conflitos
!grep -viE 'torch|torchvision' requirements.txt > /content/yolov7/req_no_torch.txt
# instalar com wheels quando possível (evita compilar)
!pip -q install -r req_no_torch.txt --only-binary=:all: || pip -q install -r req_no_torch.txt

# 4) Preparar frames (caso ainda não tenha sido feito)
%cd /content
import os, cv2
os.makedirs("/content/frames", exist_ok=True)
if len(os.listdir("/content/frames")) == 0:
    # usa os frames coletados na CEL 6 (lista 'frames') para exportar ~30 imagens
    for i, fr in enumerate(frames[:30]):
        cv2.imwrite(f"/content/frames/{i:04d}.jpg", fr)
    print("🖼️ Exportei", min(30, len(frames)), "frames para /content/frames")

# 5) Rodar detecção
%cd /content/yolov7
!python detect.py --weights yolov7.pt --conf 0.25 --source /content/frames --save-txt --project /content/y7out --name det --exist-ok

print("✅ Resultados em:")
print("  - imagens: /content/y7out/det")
print("  - labels:  /content/y7out/det/labels")


/content
⬇️ Instalando PyTorch + TorchVision (CUDA 12.1 wheels)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 MB 707.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━